In [34]:
!pip install -U pinecone sentence-transformers openai
!pip install openai --quiet
!pip install sentence-transformers

  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl (664.8 MB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl (664.8 MB)


In [35]:


from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
import openai
import pandas as pd

# 3. Pinecone and OpenAI setup
PINECONE_API_KEY = "pcsk_2czr3A_58ustdWMomnjZ7p1P3P5UaNPagR9h61WNiW6iAeXFiDkH3gWrNoqpbxFghTZYCY"
INDEX_NAME = "bangla-book-index"
REGION = "us-east-1"

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(INDEX_NAME)

model = SentenceTransformer('intfloat/multilingual-e5-large')
import os
openai.api_key =""

chat_history = []
def answer_from_book(user_query, top_k=5):
    global chat_history

    # Keep only last 10 turns
    if len(chat_history) > 10:
        chat_history = chat_history[-10:]

    # Prepare chat history string for LLM
    history_str = ""
    for turn in chat_history:
        history_str += f"User: {turn['user']}\nAssistant: {turn['assistant']}\n"

    # RAG retrieval
    query_emb = model.encode(f"query: {user_query}")
    results = index.query(
        vector=query_emb.tolist(),
        top_k=top_k,
        include_metadata=True
    )
    retrieved_chunks = [match['metadata']['text'] for match in results['matches']]
    context = "\n\n".join([f"[Passage {i+1}]\n{chunk}" for i, chunk in enumerate(retrieved_chunks)])
    #  LLM prompt

    llm_prompt = f"""

You are a Bangla QA assistant for textbooks. The question can be asked in both english or bangla but you need to answer in bangla.it in Bangla.

Below is the recent conversation history. Use it for context if needed:
{history_str}

You have been provided with several retrieved passages from a textbook. These passages may include:
- Explanatory or narrative text (main passages)
- Tables
- MCQ (multiple choice questions) and their options or answer keys
- Lists and other formatted content

**Instructions:**
1. **If the question includes the word 'বাগধারাটি', first look for an answer in the word meaning section (such as sections titled 'বাগধারা', 'বাগধারার অর্থ', or where word meanings are explained).**
2. **First, carefully read the main explanatory/narrative passages. Try to find the answer to the user's question in these main text sections.**
3. **If you do not find the answer in the passages, then check the MCQ, option lists, tables, or answer keys in the retrieved text.**
4. If you still cannot find the answer, reply only with: 'Not found in the book.' Never make up information or use outside knowledge.

If multiple sources in the passages or MCQ part provide relevant details, synthesize the answer based on all of them.


Below are some examples:

User Question: অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?
Expected Answer: শুম্ভুনাথ

User Question: কাকে অনুপমের ভাগ্য দেবতা বলে উল্লেখ করা হয়েছে?
Expected Answer: মামাকে

User Question: বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?
Expected Answer: ১৫ বছর

Here are the most relevant passages:
{context}

Answer the following question using ONLY the information in the passages above.

Question: {user_query}
"""
    # LLM call
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": "You are an expert educational QA assistant."},
            {"role": "user", "content": llm_prompt}
        ],
        temperature=0.2,
        max_tokens=512
    )
    answer = response.choices[0].message.content.strip()

    # Add turn to chat history
    chat_history.append({"user": user_query, "assistant": answer})

    # Again keep only last 10
    if len(chat_history) > 10:
        chat_history = chat_history[-10:]

    return answer




In [ ]:
# 5. Example usage:
q = " Who was Anupam?"
print(answer_from_book(q))



In [ ]:
q = " how was horish define in book?"
print(answer_from_book(q))

হরিশকে বইয়ে এমন একজন চরিত্র হিসেবে বর্ণনা করা হয়েছে যিনি সাহসী, উচ্ছল এবং রসিক স্বভাবের। উদ্দীপকে উল্লেখ আছে যে শাফিক নামের চরিত্রটি যাকে হরিশের সঙ্গে তুলনা করা হয়েছে, সে যেকোনো পরিবেশে দ্রুত নিজেকে মানিয়ে নিতে পারে এবং আলোচনার মধ্যমণি হয়ে ওঠে। এছাড়া, 'অপরিচিতা' গল্পে হরিশের গুণ হিসেবে আসর জমানোর ক্ষমতা উল্লেখ আছে। তাই হরিশকে সাহসিকতা, ব্যক্তিত্ব এবং রসিকতার সমন্বয়ে গড়ে উঠা একজন প্রাণবন্ত ও সক্রিয় চরিত্র হিসেবে দেখা যায়।


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

# question-answer pairs (can use DataFrame or dict)
qa_pairs = [
    {"question": "‘মাকাল ফল’ বাগধারাটি কী বোঝাতে ব্যবহৃত হয়েছে?", "answer": "বাহ্যিকভাবে সুন্দর কিন্তু মূল্যহীন"},
    {"question": "‘অপরিচিতা' গল্পটি প্রথম কোথায় প্রকাশিত হয়?", "answer": "সবুজপত্র"},
    {"question": "হরিশি কোথায় কর্মরত ছিলেন?", "answer": "কানপুর"},
    {"question": "অনুপমের প্রকৃত অভিভাবক কে?", "answer": "মামা"},
    {"question": "এসপাস-ওসপাস' বাগধারাটির অর্থ কী?", "answer": "একে অপরকে দোষারোপ"},
    {"question": "‘অপরিচিতা’ গল্পে অনুপমের আত্মপরিচয় কীভাবে ফুটে ওঠে?", "answer": "নীরবতা ও আত্মদ্বন্দ্বে"},
    {"question": "মন্দ নয়, খাঁটি সোনা বলতে হয়’ — এই উক্তিটি কে বলেন?", "answer": "বিনুদাদা"},
    {"question": "‘অপরিচিতা' গল্পে অনুপমের বয়স কত ছিল?", "answer": "২৭"},
    {"question": "‘অপরিচিতা’ গল্পে অনুপম কী নিয়ে দ্বিধায় ভোগে?", "answer": "আত্মসম্মান"},
    {"question": "অনুপমকে প্রথম যে মেয়েটি প্রত্যাখ্যান করে, সে কে?", "answer": "অনুপমকে প্রথম যে মেয়েটি প্রত্যাখ্যান করে, তিনি হলেন কল্যাণী।"},
    {"question": "হরিশিকে ‘অপরিচিতা’ গল্পে কীভাবে চিহ্নিত করা হয়েছে?", "answer": "রসিক মনের মানুষ"},
]


e5_model = SentenceTransformer('intfloat/multilingual-e5-large')

results = []

#compute cosine similarity
for qa in qa_pairs:
    question = qa['question']
    ground_truth = qa['answer']

    llm_answer = answer_from_book(question)
    print(f"\nQ: {question}")
    print(f"LLM Answer: {llm_answer}")
    print(f"Expected: {ground_truth}")

    # Compute semantic similarity
    emb1 = e5_model.encode(ground_truth, convert_to_tensor=True)
    emb2 = e5_model.encode(llm_answer, convert_to_tensor=True)
    score = util.pytorch_cos_sim(emb1, emb2).item()

    print(f"Cosine Similarity: {score:.4f}")

    # Collect results for DataFrame or CSV
    results.append({
        "question": question,
        "ground_truth": ground_truth,
        "llm_answer": llm_answer,
        "cosine_similarity": score
    })


df = pd.DataFrame(results)
df.to_csv("/content/drive/MyDrive/rag_eval_results.csv", index=False)


In [ ]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd


qa_pairs = [
    {"question": "‘মাকাল ফল’ বাগধারাটি কী বোঝাতে ব্যবহৃত হয়েছে?", "answer": "বাহ্যিকভাবে সুন্দর কিন্তু মূল্যহীন"},
    {"question": "‘অপরিচিতা' গল্পটি প্রথম কোথায় প্রকাশিত হয়?", "answer": "সবুজপত্র"},
    {"question": "হরিশি কোথায় কর্মরত ছিলেন?", "answer": "কানপুর"},
    {"question": "অনুপমের প্রকৃত অভিভাবক কে?", "answer": "মামা"},
    {"question": "এসপাস-ওসপাস' বাগধারাটির অর্থ কী?", "answer": "একে অপরকে দোষারোপ"},
    {"question": "‘অপরিচিতা’ গল্পে অনুপমের আত্মপরিচয় কীভাবে ফুটে ওঠে?", "answer": "নীরবতা ও আত্মদ্বন্দ্বে"},
    {"question": "মন্দ নয়, খাঁটি সোনা বলতে হয়’ — এই উক্তিটি কে বলেন?", "answer": "বিনুদাদা"},
    {"question": "‘অপরিচিতা' গল্পে অনুপমের বয়স কত ছিল?", "answer": "২৬"},
    {"question": "‘অপরিচিতা’ গল্পে অনুপম কী নিয়ে দ্বিধায় ভোগে?", "answer": "আত্মসম্মান"},
    {"question": "অনুপমকে প্রথম যে মেয়েটি প্রত্যাখ্যান করে, সে কে?", "answer": "অনুপমকে প্রথম যে মেয়েটি প্রত্যাখ্যান করে, তিনি হলেন কল্যাণী।"},
    {"question": "হরিশিকে ‘অপরিচিতা’ গল্পে কীভাবে চিহ্নিত করা হয়েছে?", "answer": "রসিক মনের মানুষ"},
]


e5_model = SentenceTransformer('intfloat/multilingual-e5-large')

def rag_relevance_eval(qa_pairs, k=5, sim_threshold=0.7):
    relevant_count = 0
    all_results = []

    for qa in qa_pairs:
        question = qa['question']
        ground_truth = qa['answer']

        # Get top-K retrieved chunks
        query_emb = e5_model.encode(f"query: {question}")
        results = index.query(
            vector=query_emb.tolist(),
            top_k=k,
            include_metadata=True
        )
        retrieved_chunks = [match['metadata']['text'] for match in results['matches']]

        # 1. String match
        found = any(ground_truth in chunk for chunk in retrieved_chunks)
        found_method = 'string' if found else None
        max_sim = 0

        # Semantic similarity if not found by string
        if not found:
            gt_emb = e5_model.encode(ground_truth, convert_to_tensor=True)
            for chunk in retrieved_chunks:
                chunk_emb = e5_model.encode(chunk, convert_to_tensor=True)
                sim = util.pytorch_cos_sim(gt_emb, chunk_emb).item()
                if sim > max_sim:
                    max_sim = sim
            if max_sim >= sim_threshold:
                found = True
                found_method = 'semantic'
            else:
                found_method = None

        if found:
            relevant_count += 1

        all_results.append({
            "question": question,
            "ground_truth": ground_truth,
            "retrieved_chunks": " ||| ".join(retrieved_chunks),
            "relevant": found,
            "found_by": found_method,
            "max_semantic_sim": max_sim if found_method == 'semantic' else 1.0 if found_method == 'string' else max_sim
        })

        print(f"Q: {question}\nFound in top-{k} chunks: {found} (method: {found_method}) | Max sim: {max_sim:.3f}")

    relevance_score = relevant_count / len(qa_pairs)
    print(f"\nRAG Retriever Relevance Score: {relevance_score:.2%} (Found answer in {relevant_count} out of {len(qa_pairs)} queries)")
    return all_results, relevance_score


results, score = rag_relevance_eval(qa_pairs, k=5)

df = pd.DataFrame(results)
csv_path = "/content/drive/MyDrive/rag_relevance_eval_results.csv"
df.to_csv(csv_path, index=False)
print(f"\nResults saved to {csv_path}")
